## Functions to compute performances 

In [1]:
import numpy as np
import scipy 
import random
import math
import csv
from scipy.stats import poisson 
from scipy.integrate import quad


In [2]:
def sigmoidal(f):
    return 1/(1+np.exp(-f))

In [20]:
def compute_coverage_insample(num_realisation, num_samples, discrete_output_matrix, x_test,
                    latent_means, latent_vars, test_data_list, transformation, alpha = None, beta = None, 
                              credible_interval = None):
    np.random.seed(1)
    if credible_interval is None:
        lower = 5
        higher = 95
    else:
        lower = credible_interval/2
        higher = 100 - credible_interval/2
        
    coverage = np.zeros((num_realisation))
    true_number_events = np.sum(discrete_output_matrix, axis =0)
    count_matrix = np.zeros((num_samples, num_realisation))

    for p in range(num_realisation):
        ## Create samples from the intensity
        for s in range(num_samples):
            
            if transformation == 'sigmoidal':
                lambda_max = np.random.gamma(alpha[p], 1/(beta[p]))
                latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                intensity = (lambda_max*sigmoidal(latent_function))[:,0]
            if transformation == 'square':
                latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                intensity = ((latent_function)**2)[:,0]
            if transformation == 'exponential':

                latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                intensity = np.exp(latent_function)[:,0]
            
            
            volume = np.trapz(intensity, x_test)
            count_matrix[s,p] = np.random.poisson(volume)

            lower_5percentile = np.percentile(count_matrix[:,p], [lower])
            upper_95percentile = np.percentile(count_matrix[:,p], [higher])
#             print('lower_5percentile', lower_5percentile)
#             print('upper_95percentile', upper_95percentile)
        if true_number_events[p] >= lower_5percentile and true_number_events[p] <= upper_95percentile:
            coverage[p] = 1.
        else:
            coverage[p] = 0.
    return np.mean(coverage, axis =0), np.std(coverage, axis =0), count_matrix

In [9]:
def compute_l_2(num_realisation, predictions, true_function, x_test, num_burnin = 0, code = 'Py'):
    mse_vector = np.zeros((num_realisation, 1))
    if code == 'mat':
        for i in range(num_realisation):
            if num_burnin >0:
                num_burnin = 1000
                mean_lambda = np.mean(predictions[:,i][0][:,num_burnin:], axis = 1)
            else:
                mean_lambda = predictions[:,i][0][:,0]
            difference_functions = (mean_lambda-true_function)**2
            mse_vector[i] = np.trapz(difference_functions, x_test)
    else:
        for i in range(num_realisation):
            difference_functions = (predictions[i][:,0]-true_function)**2
            mse_vector[i] = np.trapz(difference_functions, x_test)
    return mse_vector

In [10]:
def compute_lp(num_realisation, x_test, test_data_list, 
               transformation, latent_means = None, latent_vars = None, alpha = None, beta = None, 
               num_burnin = 0, code ='Py'):
    np.random.seed(1)
    log_prob_tensor = np.zeros((num_realisation, 1))
    for p in range(num_realisation):
        num_samples = 100
        intensity_matrix = np.zeros((x_test.shape[0], num_samples))
        log_prob_matrix = np.zeros((num_samples, 1))
        ## Create samples from the intensity
        for s in range(num_samples):
            if transformation == 'sigmoidal':
                if code == 'mat':
                    num_burnin = 1000
                    intensity_matrix = latent_means[:,p][0][:,num_burnin:]
                else:
                    lambda_max = np.random.gamma(alpha[p], 1/(beta[p]))
                    latent_function = latent_means[p,:,:] + np.sqrt(latent_vars[p,:,:])*np.random.normal(0, 1)
                    intensity_matrix[:,s] = (lambda_max*sigmoidal(latent_function))[:,0]
        
            if transformation == 'square':
                if code == 'mat':
                    posterior_mean = latent_means[:,p][0]
                    posterior_var = latent_vars[:,p][0]
                    latent_function = posterior_mean + np.sqrt(posterior_var)*np.random.normal(0, 1)
                    intensity_matrix[:,s] = ((latent_function)[:,0])**2
                else:
                    latent_function = latent_means[p,:,:] + np.sqrt(latent_vars[p,:,:])*np.random.normal(0, 1)
                    intensity_matrix[:,s] = ((latent_function)[:,0])**2
            if transformation == 'exponential':
                latent_function = latent_means[p,:,:] + np.sqrt(latent_vars[p,:,:])*np.random.normal(0, 1)
                intensity_matrix[:,s] = np.exp(latent_function)[:,0]

            volume = np.trapz(intensity_matrix[:,s], x_test)
            log_prob = -volume
            
            for i in range(10):
                test_data = test_data_list[i]
                log_prob = log_prob + np.sum(np.log(np.interp(test_data, x_test, intensity_matrix[:,s])))
            
            log_prob_matrix[s] = log_prob
    
        #print('Predictive Logprob_'+str(j), np.mean(log_prob_matrix/10))
        log_prob_tensor[p] = np.mean(log_prob_matrix/num_realisation)
    return log_prob_tensor

In [11]:
def compute_Q2(num_realisation, predictions, true_function, num_burnin = 0, code = 'Py'):
    Q_2 = np.zeros((num_realisation,1))
    if code == 'mat':
        if num_burnin >0:
            for p in range(num_realisation):
                Q_2[p] = 1. - np.mean((true_function - np.mean(predictions[:,p][0][:,num_burnin:], axis = 1))**2)/np.var(true_function)
        else:
            for p in range(num_realisation):
                Q_2[p] = 1. - np.mean((true_function - predictions[:,p][0][:,0])**2)/np.var(true_function)
    else:
        for p in range(num_realisation):
            Q_2[p] = 1. - np.mean((true_function - predictions[p][:,0])**2)/np.var(true_function)
    return Q_2

In [12]:
def compute_coverage_out_sample(num_realisation, num_samples, discrete_output_matrix1, x_test,
                    latent_means, latent_vars, test_data_list, transformation, alpha = None, beta = None,
                               credible_interval = None):
    np.random.seed(1)
    if credible_interval is None:
        lower = 5
        higher = 95
    else:
        lower = credible_interval/2
        higher = 100 - credible_interval/2
    coverage = 0.
    true_number_events = np.sum(discrete_output_matrix1, axis =0)
    count_matrix = np.zeros((num_samples, num_realisation))
    coverage_matrix = np.zeros((num_realisation, len(test_data_list)))

    for p in range(num_realisation):
        ## Create samples from the intensity
        for s in range(num_samples):
            
            if transformation == 'sigmoidal':
                lambda_max = np.random.gamma(alpha[p], 1/(beta[p]))
                latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                intensity = (lambda_max*sigmoidal(latent_function))[:,0]
            if transformation == 'square':
                latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                intensity = ((latent_function)**2)[:,0]
            if transformation == 'exponential':
                latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                intensity = np.exp(latent_function)[:,0]
            
            
            volume = np.trapz(intensity, x_test)
            count_matrix[s,p] = np.random.poisson(volume)

        lower_5percentile = np.percentile(count_matrix[:,p], [lower])
        upper_95percentile = np.percentile(count_matrix[:,p], [higher])

        for t in range(len(test_data_list)):
            test_number = test_data_list[t].shape[0]
            if test_number >= lower_5percentile and test_number <= upper_95percentile:
                coverage_matrix[p,t] = 1.
            else:
                coverage_matrix[p,t] = 0.
    return np.mean(np.sum(coverage_matrix, axis = 1)/len(test_data_list)), np.std(np.sum(coverage_matrix, axis = 1)/len(test_data_list))



In [21]:
def compute_nlpl(x_test, num_realisation, latent_means, latent_vars, 
    test_data_list, transformation, num_samples, alpha = None, beta = None, area_cell = None, code = 'Py', dim = 1):
    np.random.seed(1)
    if dim ==1:
        intensity_matrix = np.zeros((x_test.shape[0], num_samples))
        nlp_matrix = np.zeros((len(test_data_list), num_realisation))
        for p in range(num_realisation):
            for t in range(len(test_data_list)):
                log_prob = 0.
                for s in range(num_samples):
                    if transformation == 'sigmoidal':
                        if code == 'mat':
                            num_burnin = 1000
                            intensity = latent_means[:,p][0][:,num_burnin:][:,s]
                        else:
                            lambda_max = np.random.gamma(alpha[p], 1/(beta[p]))
                            latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                            intensity = (lambda_max*sigmoidal(latent_function))[:,0]

                    if transformation == 'square':
                        if code == 'mat':
                            posterior_mean = latent_means[:,p][0]
                            posterior_var = latent_vars[:,p][0]
                            latent_function = posterior_mean + np.sqrt(posterior_var)*np.random.normal(0, 1)
                            intensity = ((latent_function)[:,0])**2
                        else:
                            latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                            intensity = (latent_function**2)[:,0]
                    if transformation == 'exponential':
                        latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                        intensity = (np.exp(latent_function))[:,0]

                    volume = np.trapz(intensity, x_test)
                    log_prob += scipy.stats.poisson.logpmf(test_data_list[t].shape[0], volume)
                nlpl = log_prob/num_samples
                nlp_matrix[t, p] = -nlpl
    else:
        intensity_matrix = np.zeros((x_test.shape[0], int(num_samples)))
        nlp_matrix = np.zeros(int(num_samples))
        for p in range(num_realisation):
            for s in range(num_samples):
                if transformation == 'sigmoidal':
                    if code == 'mat':
                        num_burnin = 1000
                        intensity = latent_means[:,p][0][:,num_burnin:][:,s]
                    else:
                        lambda_max = np.random.gamma(alpha[p], 1/(beta[p]))
                        latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                        intensity = (lambda_max*sigmoidal(latent_function))[:,0]

                if transformation == 'square':
                    if code == 'mat':
                        posterior_mean = latent_means
                        posterior_var = latent_vars
                        latent_function = posterior_mean + np.sqrt(posterior_var)*np.random.normal(0, 1)
                        intensity = ((latent_function))**2
                    else:
                        latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                        intensity = (latent_function**2)
                if transformation == 'exponential':
                    latent_function = latent_means[p,:] + np.sqrt(latent_vars[p,:])*np.random.normal(0, 1)
                    intensity = (np.exp(latent_function))[:,0]

                ## Here we need to do a double integration to compute the area under the surface
                volume = np.sum((area_cell**2)*intensity)
#                 print('test_data_list.shape[0]', test_data_list.shape[0])
                log_prob = scipy.stats.poisson.logpmf(test_data_list.shape[0], volume)
                nlp_matrix[s] = -log_prob/num_samples
            #nlp_matrix = nlpl[np.newaxis]
            #print('nlp_matrix', nlp_matrix)
            
    ## Average on different test set
    return np.mean(nlp_matrix, axis = 0), np.std(nlp_matrix, axis = 0)


In [16]:
def compute_l_test(num_realisation, x_test, test_data_list, 
                   transformation, num_samples, area_cell, latent_means = None, latent_vars = None, alpha = None, beta = None,
                   dim = 2, code ='Py'):
    np.random.seed(1)
    
    #num_samples = 100
    intensity_matrix = np.zeros((x_test.shape[0], num_samples))
    log_prob_matrix = np.zeros((num_samples, 1))
    ## Create samples from the intensity
    for s in range(num_samples):
        if transformation == 'sigmoidal':
            if code == 'mat':
                num_burnin = 1000
                intensity_matrix = latent_means[:,0][0][:,num_burnin:]
            else:
                lambda_max = np.random.gamma(alpha[0], 1/(beta[0]))
                latent_function = latent_means[0,:,:] + np.sqrt(latent_vars[0,:,:])*np.random.normal(0, 1)
                intensity_matrix[:,s] = (lambda_max*sigmoidal(latent_function))[:,0]

        if transformation == 'square':
            if code == 'mat':
                posterior_mean = latent_means
                posterior_var = latent_vars
                latent_function = posterior_mean + np.sqrt(posterior_var)*np.random.normal(0, 1)
                intensity_matrix[:,s] = ((latent_function)[:,0])**2
            else:
                latent_function = latent_means[:,0] + np.sqrt(latent_vars[:,0])*np.random.normal(0, 1)
                intensity_matrix[:,s] = ((latent_function))**2
        if transformation == 'exponential':
            latent_function = latent_means[0,:,:] + np.sqrt(latent_vars[0,:,:])*np.random.normal(0, 1)
            intensity_matrix[:,s] = np.exp(latent_function)[:,0]

        volume = np.sum((area_cell**2)*intensity_matrix[:,s])
        constant_part = -volume

        ## Interpolate intensity at these data points
        location_part = np.sum(np.log(scipy.interpolate.griddata(x_test, intensity_matrix[:,s], test_data_list)))
        #print('i', intensity_matrix[:,s])
        
        
        log_prob_matrix[s] = constant_part + location_part

    log_prob_tensor = np.mean(log_prob_matrix)
   
    return np.mean(log_prob_matrix), np.std(log_prob_matrix)